In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import pickle
import torch.optim as optim
import copy

## 0. Dataset and DataLoader

In [ ]:
class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)

        self.data = self.data.iloc[::5, :].reset_index(drop=True)

        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()

        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()

            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()

            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        history_weather = self.data.iloc[idx:idx + 288, 4:12].values.astype(float)  # [288, 8]
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)   # [288]
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)  # [24, 8]
        future_wind_power = self.data.iloc[idx + 312, 2]  # float

        return (
            #torch.tensor(history_weather, dtype=torch.float32),
            torch.tensor(wind_power_history, dtype=torch.float32),
            torch.tensor(future_weather, dtype=torch.float32),
            torch.tensor(future_wind_power, dtype=torch.float32)
        )
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }
    
name='CAISO_zone_1_.csv'
with open('weather_scaler.pkl', 'rb') as f:
    weather_scaler = pickle.load(f)
dataset = WindPowerDataset(name, save_scalers=True)
wind_power_scaler=dataset.wind_power_scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=32
test_split=0.2
test_size = int(len(dataset) * test_split)
train_size = len(dataset) - test_size
train_dataset = Subset(dataset, list(range(train_size)))
test_dataset = Subset(dataset, list(range(train_size, len(dataset))))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1. preparation

In [ ]:
# This is the RUB/UP/RUPW data obtained by the defending party (wind farm) through training their own wind power prediction model. (based on target model, not surrogate model)
uap_loaded_list = [
    torch.load('RUP_caiso_003.pth').to('cpu'),
    torch.load('RUP_caiso_005.pth').to('cpu'),
    torch.load('RUP_caiso_010.pth').to('cpu'),
    torch.load('UP_caiso_003.pth').to('cpu'),
    torch.load('UP_caiso_005.pth').to('cpu'),
    torch.load('UP_caiso_010.pth').to('cpu'),
    torch.load('RUPW_caiso_003.pth').to('cpu'),
    torch.load('RUPW_caiso_005.pth').to('cpu'),
    torch.load('RUPW_caiso_010.pth').to('cpu')
]

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
def load_model(model_path, device='cpu'):
    model =WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path = 'wind_caiso_lstm_sigmoid_version2.pth' 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(model_path, device)

## 2. PALM

In [ ]:
def compute_constraints(weather_future, weather_scaler):
    
    batch_size, seq_len, feature_dim = weather_future.shape

    # Denormalization
    weather_future_reshaped = weather_future.view(-1, feature_dim)
    weather_future_unscaled = weather_scaler.inverse_transform(weather_future_reshaped.detach().cpu().numpy())
    weather_future_unscaled = torch.tensor(weather_future_unscaled, dtype=weather_future.dtype, device=weather_future.device)
    weather_future_unscaled = weather_future_unscaled.view(batch_size, seq_len, feature_dim)

    # extract variables
    DNI = weather_future_unscaled[:, :, 1]
    Solar_Zenith_Angle = weather_future_unscaled[:, :, 4]
    DHI = weather_future_unscaled[:, :, 0]
    GHI = weather_future_unscaled[:, :, 2]
    Dew_Point = weather_future_unscaled[:, :, 3]
    Temperature = weather_future_unscaled[:, :, 7]
    Relative_Humidity = weather_future_unscaled[:, :, 6]

    # Physical constraints
    cos_sza = torch.cos(Solar_Zenith_Angle * np.pi / 180.0)
    constraint_1 = torch.abs((DNI * cos_sza + DHI) - GHI)  # GHI = DNI*cos + DHI
    constraint_2 = torch.relu(Dew_Point - Temperature)     # Dew point <= Temperature
    constraint_3 = torch.abs(
        100 * torch.exp(17.625 * Dew_Point / (243.04 + Dew_Point)) /
        torch.exp(17.625 * Temperature / (243.04 + Temperature)) - Relative_Humidity
    )

    return torch.stack([constraint_1, constraint_2, constraint_3], dim=-1)


# define the classifier network (Normal=1, Adversarial=0)
class ConstraintClassifier(nn.Module):
    def __init__(self, input_dim=72):
        super(ConstraintClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x


# Train Classifier
def train_constraint_classifier_with_loader(
    train_loader,
    uap_loaded_list,
    classifier_model,
    wind_power_model,
    weather_scaler,
    device='cuda',
    epochs=10,
    learning_rate=1e-5
):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(classifier_model.parameters(), lr=learning_rate)
    classifier_model.to(device)
    wind_power_model.to(device)

    # Freeze power prediction model parameters
    wind_power_model.eval()
    for param in wind_power_model.parameters():
        param.requires_grad = False

    best_loss_value = float('inf')
    epsilons = [0.03, 0.05, 0.10]  # different perturbation strengths

    print(f" Training with epsilon values: {epsilons}")
    print(" Label: Normal = 1, Adversarial = 0")

    for epoch in range(epochs):
        total_loss = 0.0
        total_steps = 0
        classifier_model.train()

        for batch_idx, (wind_history, weather_future, future_wind_power) in enumerate(train_loader):
            B = weather_future.size(0)
            weather_future = weather_future.to(device)
            wind_history = wind_history.to(device)
            future_wind_power = future_wind_power.to(device)

            # Store constraints and labels for all samples
            all_constraints = []
            all_labels = []

            with torch.no_grad():
                normal_constraints = compute_constraints(weather_future.cpu(), weather_scaler).to(device)
                normal_constraints = normal_constraints.view(B, -1)
                all_constraints.append(normal_constraints)
                all_labels.append(torch.ones(B, device=device))  # normal = 1

            for uap in uap_loaded_list:
                adv_weather = torch.clamp(weather_future + uap.to(device), 0, 1)
                attacked_constraints = compute_constraints(adv_weather.cpu(), weather_scaler).to(device)
                attacked_constraints = attacked_constraints.view(B, -1)
                all_constraints.append(attacked_constraints)
                all_labels.append(torch.zeros(B, device=device))  # abnormal = 0

            wind_power_model.train()  # must be in train() mode

            # Ensure wind_power_model parameters are not updated
            for param in wind_power_model.parameters():
                param.requires_grad = False  # freeze parameters

            weather_future_for_grad = weather_future.clone().detach().requires_grad_(True)
            pred_power = wind_power_model(wind_history, weather_future_for_grad)
            loss_power = nn.MSELoss()(pred_power.squeeze(), future_wind_power)

            # backward
            wind_power_model.zero_grad()
            loss_power.backward()  # now can backward

            # gain the gradient
            grad_sign = weather_future_for_grad.grad.data.sign()

            # Generate adversarial samples with multiple epsilons
            for eps in epsilons:
                adv_weather = torch.clamp(weather_future + eps * grad_sign, 0, 1)
                attacked_constraints = compute_constraints(adv_weather.cpu(), weather_scaler).to(device)
                attacked_constraints = attacked_constraints.view(B, -1)
                all_constraints.append(attacked_constraints)
                all_labels.append(torch.zeros(B, device=device))  # abnormal = 0

            # Restore eval() (optional)
            wind_power_model.eval()

            # Concatenate all samples
            combined_constraints = torch.cat(all_constraints, dim=0)
            combined_labels = torch.cat(all_labels, dim=0)

            # Forward pass
            outputs = classifier_model(combined_constraints).squeeze()
            loss = criterion(outputs, combined_labels)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_steps += 1

        avg_loss = total_loss / total_steps
        print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.6f}")

        # save best model
        if avg_loss < best_loss_value:
            best_loss_value = avg_loss
            best_classifier_model = copy.deepcopy(classifier_model)
            print(f"✅ Best model updated with loss: {best_loss_value:.6f}")

    print(f"🎉 Training completed. Best loss: {best_loss_value:.6f}")
    return best_classifier_model

In [ ]:
# initialize model
classifier_model = ConstraintClassifier(input_dim=72)

# start training
trained_classifier = train_constraint_classifier_with_loader(
    train_loader=train_loader,
    uap_loaded_list=uap_loaded_list,
    classifier_model=classifier_model,
    wind_power_model=model,
    weather_scaler=weather_scaler,
    device='cuda',
    epochs=30,
    learning_rate=1e-5
)

torch.save(trained_classifier,'CAISO_zone_1_PALM.pth')